In [ ]:
!pip install gensim==4.1.2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!unzip /content/drive/MyDrive/data.zip

Archive:  /content/drive/MyDrive/data.zip
  inflating: data/testing_corpus.csv  
  inflating: data/testing_queries.csv  
  inflating: data/testing_result.csv  
  inflating: data/training_corpus.csv  
  inflating: data/training_queries.csv  
  inflating: data/training_result.csv  


In [ ]:
import pandas as pd
import numpy as np
import spacy

In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm',disable=['ner','parser'])
nlp.max_length=5000000

In [ ]:
cd data

/content/data/data


In [ ]:
training_corpus=pd.read_csv("training_corpus.csv")

In [ ]:
testing_corpus=pd.read_csv("testing_corpus.csv")

In [ ]:
training_queries=pd.read_csv("training_queries.csv")

In [ ]:
testing_queries=pd.read_csv("testing_queries.csv")

In [ ]:
training_queries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   qid     1000 non-null   int64 
 1   query   1000 non-null   object
dtypes: int64(1), object(1)
memory usage: 15.8+ KB


In [ ]:
testing_queries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   qid     1000 non-null   int64 
 1   query   1000 non-null   object
dtypes: int64(1), object(1)
memory usage: 15.8+ KB


In [ ]:
import re

# Lowercasing the text
training_corpus['cleaned']=training_corpus['body'].apply(lambda x:x.lower())
testing_corpus['cleaned']=testing_corpus['body'].apply(lambda x:x.lower())

# Dictionary of english Contractions
contractions_dict = { "ain't": "are not","'s":" is","aren't": "are not","can't": "can not","can't've": "cannot have",
"'cause": "because","could've": "could have","couldn't": "could not","couldn't've": "could not have",
"didn't": "did not","doesn't": "does not","don't": "do not","hadn't": "had not","hadn't've": "had not have",
"hasn't": "has not","haven't": "have not","he'd": "he would","he'd've": "he would have","he'll": "he will",
"he'll've": "he will have","how'd": "how did","how'd'y": "how do you","how'll": "how will","i'd": "i would",
"i'd've": "i would have","i'll": "i will","i'll've": "i will have","i'm": "i am","i've": "i have",
"isn't": "is not","it'd": "it would","it'd've": "it would have","it'll": "it will","it'll've": "it will have",
"let's": "let us","ma'am": "madam","mayn't": "may not","might've": "might have","mightn't": "might not",
"mightn't've": "might not have","must've": "must have","mustn't": "must not","mustn't've": "must not have",
"needn't": "need not","needn't've": "need not have","o'clock": "of the clock","oughtn't": "ought not",
"oughtn't've": "ought not have","shan't": "shall not","sha'n't": "shall not",
"shan't've": "shall not have","she'd": "she would","she'd've": "she would have","she'll": "she will",
"she'll've": "she will have","should've": "should have","shouldn't": "should not",
"shouldn't've": "should not have","so've": "so have","that'd": "that would","that'd've": "that would have",
"there'd": "there would","there'd've": "there would have",
"they'd": "they would","they'd've": "they would have","they'll": "they will","they'll've": "they will have",
"they're": "they are","they've": "they have","to've": "to have","wasn't": "was not","we'd": "we would",
"we'd've": "we would have","we'll": "we will","we'll've": "we will have","we're": "we are","we've": "we have",
"weren't": "were not","what'll": "what will","what'll've": "what will have","what're": "what are",
"what've": "what have","when've": "when have","where'd": "where did",
"where've": "where have","who'll": "who will","who'll've": "who will have","who've": "who have",
"why've": "why have","will've": "will have","won't": "will not","won't've": "will not have",
"would've": "would have","wouldn't": "would not","wouldn't've": "would not have","y'all": "you all",
"y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
"you'd": "you would","you'd've": "you would have","you'll": "you will","you'll've": "you will have",
"you're": "you are","you've": "you have"}

# Regular expression for finding contractions
contractions_re=re.compile('(%s)' % '|'.join(contractions_dict.keys()))

# Function for expanding contractions
def expand_contractions(text,contractions_dict=contractions_dict):
    def replace(match):
        return contractions_dict[match.group(0)]
    return contractions_re.sub(replace, text)


In [ ]:
def clean_text(text):
    text=re.sub('\w*\d\w*','', text)
    text=re.sub('\n',' ',text)
    text=re.sub(r"http\S+", "", text)
    text=re.sub('[^a-z]',' ',text)
    return text


In [ ]:
# Lowercasing the text
training_queries['cleaned']=training_queries['query'].apply(lambda x:x.lower())
testing_queries['cleaned']=testing_queries['query'].apply(lambda x:x.lower())

# Expanding contractions
training_queries['cleaned']=training_queries['cleaned'].apply(lambda x:expand_contractions(x))
testing_queries['cleaned']=testing_queries['cleaned'].apply(lambda x:expand_contractions(x))

# Cleaning queries using RegEx
training_queries['cleaned']=training_queries['cleaned'].apply(lambda x: clean_text(x))
testing_queries['cleaned']=testing_queries['cleaned'].apply(lambda x: clean_text(x))

# Removing extra spaces
training_queries['cleaned']=training_queries['cleaned'].apply(lambda x: re.sub(' +',' ',x))
testing_queries['cleaned']=testing_queries['cleaned'].apply(lambda x: re.sub(' +',' ',x))

In [ ]:
# Combining corpus and queries for training
combined_training=pd.concat([training_corpus.rename(columns={'lemmatized':'text'})['text'],training_queries.rename(columns={'cleaned':'text'})['text']]).sample(frac=1).reset_index(drop=True)

In [ ]:
from gensim.models import Word2Vec
# Creating data for the model training
train_data=[]
for i in combined_training:
    train_data.append(i.split())

# Training a word2vec model from the given data set
w2v_model = Word2Vec(train_data, vector_size=300, min_count=2,window=5, sg=1,workers=4)

In [ ]:
# Vocabulary size
print('Vocabulary size:', len(w2v_model.wv))

Vocabulary size: 198281


In [ ]:
# Function returning vector reperesentation of a document
def get_embedding_w2v(doc_tokens):
    embeddings = []
    if len(doc_tokens)<1:
        return np.zeros(300)
    else:
        for tok in doc_tokens:
            if tok in w2v_model.wv.key_to_index  :

                embeddings.append(w2v_model.wv.get_vector(tok))
            else:
                embeddings.append(np.random.rand(300))
        # mean the vectors of individual words to get the vector of the document
        return np.mean(embeddings, axis=0)

# Getting Word2Vec Vectors for Testing Corpus and Queries
testing_corpus['vector']=testing_corpus['lemmatized'].apply(lambda x :get_embedding_w2v(x.split()))
testing_queries['vector']=testing_queries['cleaned'].apply(lambda x :get_embedding_w2v(x.split()))

In [ ]:
testing_result=pd.read_csv("testing_result.csv")
training_result=pd.read_csv("training_result.csv")

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances
# Function for calculating average precision for a query
def average_precision(qid,qvector):

  # Getting the ground truth and document vectors
  qresult=testing_result.loc[testing_result['qid']==qid,['docid','rel']]
  qcorpus=testing_corpus.loc[testing_corpus['docid'].isin(qresult['docid']),['docid','vector']]
  qresult=pd.merge(qresult,qcorpus,on='docid')

  # Ranking documents for the query
  qresult['similarity']=qresult['vector'].apply(lambda x: cosine_similarity(np.array(qvector).reshape(1, -1),np.array(x).reshape(1, -1)).item())
  qresult['euclidean_distances']=qresult['vector'].apply(lambda x: euclidean_distances(np.array(qvector).reshape(1, -1),np.array(x).reshape(1, -1)).item())

  qresult.sort_values(by='similarity',ascending=False,inplace=True)

  # Taking Top 10 documents for the evaluation
  ranking=qresult.head(10)['rel'].values

  # Calculating precision
  precision=[]
  for i in range(1,11):
    if ranking[i-1]:
      precision.append(np.sum(ranking[:i])/i)

  # If no relevant document in list then return 0
  if precision==[]:
    return 0

  return np.mean(precision)

# Calculating average precision for all queries in the test set
testing_queries['AP']=testing_queries.apply(lambda x: average_precision(x['qid'],x['vector']),axis=1)

# Finding Mean Average Precision
print('Mean Average Precision=>',testing_queries['AP'].mean())

Mean Average Precision=> 0.794923091143865


In [ ]:
def ranking_ir(query):

  # pre-process Query
  query=query.lower()
  query=expand_contractions(query)
  query=clean_text(query)
  query=re.sub(' +',' ',query)

  # generating vector
  vector=get_embedding_w2v(query.split()).reshape(1, -1)
  #vector = pca.transform(d)
  # ranking documents
  documents=testing_corpus[['docid','title','body']].copy()
  documents['similarity']=testing_corpus['vector'].apply(lambda x: cosine_similarity(np.array(vector).reshape(1, -1),np.array(x).reshape(1, -1)).item())
  documents['euclidean_distances']=testing_corpus['vector'].apply(lambda x: euclidean_distances(np.array(vector).reshape(1, -1),np.array(x).reshape(1, -1)).item())

  documents.sort_values(by='similarity',ascending=False,inplace=True)

  return documents.head(10).reset_index(drop=True)

In [ ]:
ranking_ir('michael jordan')

,docid,title,body,similarity,euclidean_distances
0,D2962495,How Much Is Air Jordan Net Worth,"Profession: Basketball player, Actor, Athlete,...",0.860086,2.465790
1,D2218024,What draft round was Michael Jordan picked?,Answers.com ® Wiki Answers ® Categories Entert...,0.791858,2.648050
2,D3140874,Michael Jordan Net Worth,"Michael Jordan Net Worth· In Athletes, NBAMich...",0.787886,2.770895
3,D1656700,What nba team did Michael Jordan get drafted to?,What nba team did Michael Jordan get drafted t...,0.787663,2.682195
4,D2796474,How many championships does Michael Jordan have?,Jim Schiller 1 Contribution How many champions...,0.783006,2.677405
5,D842657,How many NBA championships did Michael Jordan ...,Answers.com ® Wiki Answers ® Categories Sports...,0.773496,2.731450
6,D3492208,How many NBA championships did Michael Jordan ...,Flyinryan 16 Contributions How many NBA champi...,0.772770,2.727310
7,D2785629,What is the name of a basketball team that is ...,Answers.com ® Wiki Answers ® Categories Entert...,0.768710,2.785793
8,D2688388,How many career points did Michael Jordan score?,Yaaaaa23 2 Contributions How many career point...,0.757738,2.814835
9,D1900195,What year did Michael Jordan play baseball?,Answers.com ® Wiki Answers ® Categories Entert...,0.752835,2.861421


In [ ]:
ranking_ir('nba championship michael jordan')

,docid,title,body,similarity,euclidean_distances
0,D2796474,How many championships does Michael Jordan have?,Jim Schiller 1 Contribution How many champions...,0.917741,1.769149
1,D842657,How many NBA championships did Michael Jordan ...,Answers.com ® Wiki Answers ® Categories Sports...,0.912429,1.832029
2,D3492208,How many NBA championships did Michael Jordan ...,Flyinryan 16 Contributions How many NBA champi...,0.877617,1.999296
3,D2962495,How Much Is Air Jordan Net Worth,"Profession: Basketball player, Actor, Athlete,...",0.857512,2.189376
4,D2785629,What is the name of a basketball team that is ...,Answers.com ® Wiki Answers ® Categories Entert...,0.852926,2.178668
5,D2218024,What draft round was Michael Jordan picked?,Answers.com ® Wiki Answers ® Categories Entert...,0.849518,2.132557
6,D1656700,What nba team did Michael Jordan get drafted to?,What nba team did Michael Jordan get drafted t...,0.846635,2.165955
7,D2688388,How many career points did Michael Jordan score?,Yaaaaa23 2 Contributions How many career point...,0.843719,2.202712
8,D3140874,Michael Jordan Net Worth,"Michael Jordan Net Worth· In Athletes, NBAMich...",0.827994,2.341687
9,D1900195,What year did Michael Jordan play baseball?,Answers.com ® Wiki Answers ® Categories Entert...,0.821809,2.331161


In [ ]:
# Function returning vector reperesentation of a document
def get_embedding_w2v(doc_tokens):
    embeddings = []
    if len(doc_tokens)<1:
        return np.zeros(300)
    else:
        for tok in doc_tokens:
            if tok in w2v_model.wv.key_to_index  :

                embeddings.append(w2v_model.wv.get_vector(tok))
            else:
                embeddings.append(np.random.rand(300))
        # mean the vectors of individual words to get the vector of the document
        return np.mean(embeddings, axis=0)

# Getting Word2Vec Vectors for Testing Corpus and Queries
testing_corpus['vector']=testing_corpus['lemmatized'].apply(lambda x :get_embedding_w2v(x.split()))
testing_queries['vector']=testing_queries['cleaned'].apply(lambda x :get_embedding_w2v(x.split()))

In [ ]:
a = np.zeros((testing_corpus['vector'].shape[0],testing_corpus['vector'][0].shape[0]))
for i in range(len(testing_corpus['vector'])):
    a[i,:] = testing_corpus['vector'][i]

b = np.zeros((testing_queries['vector'].shape[0],testing_queries['vector'][0].shape[0]))
for i in range(len(testing_queries['vector'])):
    b[i,:] = testing_queries['vector'][i]

from sklearn.decomposition import PCA
from sklearn.decomposition import FactorAnalysis

pca1 = PCA(n_components=300)
aa = pca1.fit_transform(a)
#pca1 = PCA(n_components = 300)
bb = pca1.transform(b)

pca = FactorAnalysis(n_components=290)
testing_corpus1 = pca.fit_transform(aa)
#pca1 = PCA(n_components = 300)
testing_queries1 = pca.transform(bb)

In [ ]:
for i in range(len(testing_corpus['vector'])):
     testing_corpus['vector'][i]=testing_corpus1[i]
for i in range(len(testing_queries['vector'])):
     testing_queries['vector'][i]=testing_queries1[i]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances
# Function for calculating average precision for a query
def average_precision(qid,qvector):

  # Getting the ground truth and document vectors
  qresult=testing_result.loc[testing_result['qid']==qid,['docid','rel']]
  qcorpus=testing_corpus.loc[testing_corpus['docid'].isin(qresult['docid']),['docid','vector']]
  qresult=pd.merge(qresult,qcorpus,on='docid')

  # Ranking documents for the query
  qresult['similarity']=qresult['vector'].apply(lambda x: cosine_similarity(np.array(qvector).reshape(1, -1),np.array(x).reshape(1, -1)).item())
  qresult['euclidean_distances']=qresult['vector'].apply(lambda x: euclidean_distances(np.array(qvector).reshape(1, -1),np.array(x).reshape(1, -1)).item())

  qresult.sort_values(by='similarity',ascending=False,inplace=True)

  # Taking Top 10 documents for the evaluation
  ranking=qresult.head(10)['rel'].values

  # Calculating precision
  precision=[]
  for i in range(1,11):
    if ranking[i-1]:
      precision.append(np.sum(ranking[:i])/i)

  # If no relevant document in list then return 0
  if precision==[]:
    return 0

  return np.mean(precision)

# Calculating average precision for all queries in the test set
testing_queries['AP']=testing_queries.apply(lambda x: average_precision(x['qid'],x['vector']),axis=1)

# Finding Mean Average Precision
print('Mean Average Precision=>',testing_queries['AP'].mean())

Mean Average Precision=> 1.0


In [ ]:
def ranking_ir(query):

  # pre-process Query
  query=query.lower()
  query=expand_contractions(query)
  query=clean_text(query)
  query=re.sub(' +',' ',query)

  # generating vector
  d=get_embedding_w2v(query.split()).reshape(1, -1)
  d=pca1.transform(d)
  vector = pca.transform(d)
  # ranking documents
  documents=testing_corpus[['docid','title','body']].copy()
  documents['similarity']=testing_corpus['vector'].apply(lambda x: cosine_similarity(np.array(vector).reshape(1, -1),np.array(x).reshape(1, -1)).item())
  documents['euclidean_distances']=testing_corpus['vector'].apply(lambda x: euclidean_distances(np.array(vector).reshape(1, -1),np.array(x).reshape(1, -1)).item())

  documents.sort_values(by='similarity',ascending=False,inplace=True)

  return documents.head(10).reset_index(drop=True)

In [ ]:
ranking_ir('michael jordan')

,docid,title,body,similarity,euclidean_distances
0,D1911483,My Doctor Online The Permanente Medical Group,Abnormal Vaginal Bleeding in Midlife and Beyon...,0.0,0.0
1,D660994,Cecilia Meaning,Cecilia Meaning Name: What does Cecilia mean? ...,0.0,0.0
2,D872248,"Category:Doll Babies, Voodoo Dolls, and Poppets","Category: Doll Babies, Voodoo Dolls, and Poppe...",0.0,0.0
3,D600451,"Weeki Wachee, FL Real Estate & Homes for Sale","Weeki Wachee, FL Real Estate & Homes for Sale1...",0.0,0.0
4,D3390302,Whatâs My IQ Test Answers,What’s My IQ Test Answers Posted on 2013/05/15...,0.0,0.0
5,D1347981,Gov Chapter 3,150 terms argentinaprincesa Gov Chapter 3Learn...,0.0,0.0
6,D221492,Intropaste,Intropaste Generic Name: barium sulfate Dosage...,0.0,0.0
7,D2194914,What sounds can we hear?,What sounds can we hear? Humans don’t hear all...,0.0,0.0
8,D408219,The Walking Dead (TV series),"From Wikipedia, the free encyclopedianavigatio...",0.0,0.0
9,D1156818,The Unofficial 2013 NBA Player Census,The Unofficial 2013 NBA Player Census October ...,0.0,0.0


In [ ]:
ranking_ir('nba championship michael jordan')

,docid,title,body,similarity,euclidean_distances
0,D1911483,My Doctor Online The Permanente Medical Group,Abnormal Vaginal Bleeding in Midlife and Beyon...,0.0,0.0
1,D660994,Cecilia Meaning,Cecilia Meaning Name: What does Cecilia mean? ...,0.0,0.0
2,D872248,"Category:Doll Babies, Voodoo Dolls, and Poppets","Category: Doll Babies, Voodoo Dolls, and Poppe...",0.0,0.0
3,D600451,"Weeki Wachee, FL Real Estate & Homes for Sale","Weeki Wachee, FL Real Estate & Homes for Sale1...",0.0,0.0
4,D3390302,Whatâs My IQ Test Answers,What’s My IQ Test Answers Posted on 2013/05/15...,0.0,0.0
5,D1347981,Gov Chapter 3,150 terms argentinaprincesa Gov Chapter 3Learn...,0.0,0.0
6,D221492,Intropaste,Intropaste Generic Name: barium sulfate Dosage...,0.0,0.0
7,D2194914,What sounds can we hear?,What sounds can we hear? Humans don’t hear all...,0.0,0.0
8,D408219,The Walking Dead (TV series),"From Wikipedia, the free encyclopedianavigatio...",0.0,0.0
9,D1156818,The Unofficial 2013 NBA Player Census,The Unofficial 2013 NBA Player Census October ...,0.0,0.0


In [ ]:
# Function returning vector reperesentation of a document
def get_embedding_w2v(doc_tokens):
    embeddings = []
    if len(doc_tokens)<1:
        return np.zeros(300)
    else:
        for tok in doc_tokens:
            if tok in w2v_model.wv.key_to_index  :

                embeddings.append(w2v_model.wv.get_vector(tok))
            else:
                embeddings.append(np.random.rand(300))
        # mean the vectors of individual words to get the vector of the document
        return np.mean(embeddings, axis=0)

# Getting Word2Vec Vectors for Testing Corpus and Queries
testing_corpus['vector']=testing_corpus['lemmatized'].apply(lambda x :get_embedding_w2v(x.split()))
testing_queries['vector']=testing_queries['cleaned'].apply(lambda x :get_embedding_w2v(x.split()))

In [ ]:
a = np.zeros((testing_corpus['vector'].shape[0],testing_corpus['vector'][0].shape[0]))
for i in range(len(testing_corpus['vector'])):
    a[i,:] = testing_corpus['vector'][i]

b = np.zeros((testing_queries['vector'].shape[0],testing_queries['vector'][0].shape[0]))
for i in range(len(testing_queries['vector'])):
    b[i,:] = testing_queries['vector'][i]

from sklearn.decomposition import PCA
from sklearn.decomposition import FactorAnalysis

pca1 = FactorAnalysis(n_components=290)
aa = pca1.fit_transform(a)
#pca1 = PCA(n_components = 300)
bb = pca1.transform(b)

pca = PCA(n_components=290)
testing_corpus1 = pca.fit_transform(aa)
#pca1 = PCA(n_components = 300)
testing_queries1 = pca.transform(bb)

In [ ]:
for i in range(len(testing_corpus['vector'])):
     testing_corpus['vector'][i]=testing_corpus1[i]
for i in range(len(testing_queries['vector'])):
     testing_queries['vector'][i]=testing_queries1[i]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances
# Function for calculating average precision for a query
def average_precision(qid,qvector):

  # Getting the ground truth and document vectors
  qresult=testing_result.loc[testing_result['qid']==qid,['docid','rel']]
  qcorpus=testing_corpus.loc[testing_corpus['docid'].isin(qresult['docid']),['docid','vector']]
  qresult=pd.merge(qresult,qcorpus,on='docid')

  # Ranking documents for the query
  qresult['similarity']=qresult['vector'].apply(lambda x: cosine_similarity(np.array(qvector).reshape(1, -1),np.array(x).reshape(1, -1)).item())
  qresult['euclidean_distances']=qresult['vector'].apply(lambda x: euclidean_distances(np.array(qvector).reshape(1, -1),np.array(x).reshape(1, -1)).item())

  qresult.sort_values(by='similarity',ascending=False,inplace=True)

  # Taking Top 10 documents for the evaluation
  ranking=qresult.head(10)['rel'].values

  # Calculating precision
  precision=[]
  for i in range(1,11):
    if ranking[i-1]:
      precision.append(np.sum(ranking[:i])/i)

  # If no relevant document in list then return 0
  if precision==[]:
    return 0

  return np.mean(precision)

# Calculating average precision for all queries in the test set
testing_queries['AP']=testing_queries.apply(lambda x: average_precision(x['qid'],x['vector']),axis=1)

# Finding Mean Average Precision
print('Mean Average Precision=>',testing_queries['AP'].mean())

Mean Average Precision=> 0.799007420005039


In [ ]:
def ranking_ir(query):

  # pre-process Query
  query=query.lower()
  query=expand_contractions(query)
  query=clean_text(query)
  query=re.sub(' +',' ',query)

  # generating vector
  d=get_embedding_w2v(query.split()).reshape(1, -1)
  d=pca1.transform(d)
  vector = pca.transform(d)
  # ranking documents
  documents=testing_corpus[['docid','title','body']].copy()
  documents['similarity']=testing_corpus['vector'].apply(lambda x: cosine_similarity(np.array(vector).reshape(1, -1),np.array(x).reshape(1, -1)).item())
  documents['euclidean_distances']=testing_corpus['vector'].apply(lambda x: euclidean_distances(np.array(vector).reshape(1, -1),np.array(x).reshape(1, -1)).item())

  documents.sort_values(by='similarity',ascending=False,inplace=True)

  return documents.head(10).reset_index(drop=True)

In [ ]:
ranking_ir('michael jordan')

,docid,title,body,similarity,euclidean_distances
0,D2962495,How Much Is Air Jordan Net Worth,"Profession: Basketball player, Actor, Athlete,...",0.965984,50.062315
1,D3140874,Michael Jordan Net Worth,"Michael Jordan Net Worth· In Athletes, NBAMich...",0.860971,56.633678
2,D1520298,Whatâs Michael Jordan Doing Now â 2018 Update,Sports What’s Michael Jordan Doing Now – 2018 ...,0.765833,64.259378
3,D2785629,What is the name of a basketball team that is ...,Answers.com ® Wiki Answers ® Categories Entert...,0.757132,58.162208
4,D2986655,Michael Jordan Biography,Quick Facts Name Michael Jordan Occupation Fam...,0.749074,65.225167
5,D1415956,Michael Jordan Biography,Quick Facts Name Michael Jordan Occupation Fam...,0.748597,65.230590
6,D848759,What Numbers Did Michael Jordan Wear?,Sports & Active Lifestyle Classic Sports Baske...,0.739694,62.153462
7,D207390,Michael Jordan: King Of The Court,Michael Jordan: King Of The Court Length: 1071...,0.738884,64.949232
8,D1142830,The 23 Best Air Jordan sneakers of All-Time,"December 20, 20144 Comments 818.2K Views The 2...",0.737081,58.198803
9,D3492208,How many NBA championships did Michael Jordan ...,Flyinryan 16 Contributions How many NBA champi...,0.730229,57.525334


In [ ]:
ranking_ir('nba championship michael jordan')

,docid,title,body,similarity,euclidean_distances
0,D842657,How many NBA championships did Michael Jordan ...,Answers.com ® Wiki Answers ® Categories Sports...,0.908367,34.055816
1,D2796474,How many championships does Michael Jordan have?,Jim Schiller 1 Contribution How many champions...,0.875295,33.474778
2,D2986655,Michael Jordan Biography,Quick Facts Name Michael Jordan Occupation Fam...,0.829060,46.002763
3,D1415956,Michael Jordan Biography,Quick Facts Name Michael Jordan Occupation Fam...,0.828019,46.016488
4,D1520298,Whatâs Michael Jordan Doing Now â 2018 Update,Sports What’s Michael Jordan Doing Now – 2018 ...,0.824911,45.267304
5,D2962495,How Much Is Air Jordan Net Worth,"Profession: Basketball player, Actor, Athlete,...",0.814347,37.957346
6,D3492208,How many NBA championships did Michael Jordan ...,Flyinryan 16 Contributions How many NBA champi...,0.797237,38.076600
7,D207390,Michael Jordan: King Of The Court,Michael Jordan: King Of The Court Length: 1071...,0.782447,46.233909
8,D3140874,Michael Jordan Net Worth,"Michael Jordan Net Worth· In Athletes, NBAMich...",0.779706,41.170238
9,D2785629,What is the name of a basketball team that is ...,Answers.com ® Wiki Answers ® Categories Entert...,0.770107,40.365922


In [ ]:
# Function returning vector reperesentation of a document
def get_embedding_w2v(doc_tokens):
    embeddings = []
    if len(doc_tokens)<1:
        return np.zeros(300)
    else:
        for tok in doc_tokens:
            if tok in w2v_model.wv.key_to_index  :

                embeddings.append(w2v_model.wv.get_vector(tok))
            else:
                embeddings.append(np.random.rand(300))
        # mean the vectors of individual words to get the vector of the document
        return np.mean(embeddings, axis=0)

# Getting Word2Vec Vectors for Testing Corpus and Queries
testing_corpus['vector']=testing_corpus['lemmatized'].apply(lambda x :get_embedding_w2v(x.split()))
testing_queries['vector']=testing_queries['cleaned'].apply(lambda x :get_embedding_w2v(x.split()))

In [ ]:
a = np.zeros((testing_corpus['vector'].shape[0],testing_corpus['vector'][0].shape[0]))
for i in range(len(testing_corpus['vector'])):
    a[i,:] = testing_corpus['vector'][i]

b = np.zeros((testing_queries['vector'].shape[0],testing_queries['vector'][0].shape[0]))
for i in range(len(testing_queries['vector'])):
    b[i,:] = testing_queries['vector'][i]

from sklearn.decomposition import PCA
from sklearn.decomposition import FactorAnalysis

pca1 = FactorAnalysis(n_components=290)
aa = pca1.fit_transform(a)
#pca1 = PCA(n_components = 300)
bb = pca1.transform(b)

pca = FactorAnalysis(n_components=290)
testing_corpus1 = pca.fit_transform(aa)
#pca1 = PCA(n_components = 300)
testing_queries1 = pca.transform(bb)

In [ ]:
for i in range(len(testing_corpus['vector'])):
     testing_corpus['vector'][i]=testing_corpus1[i]
for i in range(len(testing_queries['vector'])):
     testing_queries['vector'][i]=testing_queries1[i]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances
# Function for calculating average precision for a query
def average_precision(qid,qvector):

  # Getting the ground truth and document vectors
  qresult=testing_result.loc[testing_result['qid']==qid,['docid','rel']]
  qcorpus=testing_corpus.loc[testing_corpus['docid'].isin(qresult['docid']),['docid','vector']]
  qresult=pd.merge(qresult,qcorpus,on='docid')

  # Ranking documents for the query
  qresult['similarity']=qresult['vector'].apply(lambda x: cosine_similarity(np.array(qvector).reshape(1, -1),np.array(x).reshape(1, -1)).item())
  qresult['euclidean_distances']=qresult['vector'].apply(lambda x: euclidean_distances(np.array(qvector).reshape(1, -1),np.array(x).reshape(1, -1)).item())

  qresult.sort_values(by='similarity',ascending=False,inplace=True)

  # Taking Top 10 documents for the evaluation
  ranking=qresult.head(10)['rel'].values

  # Calculating precision
  precision=[]
  for i in range(1,11):
    if ranking[i-1]:
      precision.append(np.sum(ranking[:i])/i)

  # If no relevant document in list then return 0
  if precision==[]:
    return 0

  return np.mean(precision)

# Calculating average precision for all queries in the test set
testing_queries['AP']=testing_queries.apply(lambda x: average_precision(x['qid'],x['vector']),axis=1)

# Finding Mean Average Precision
print('Mean Average Precision=>',testing_queries['AP'].mean())

Mean Average Precision=> 0.7339466318657085


In [ ]:
def ranking_ir(query):

  # pre-process Query
  query=query.lower()
  query=expand_contractions(query)
  query=clean_text(query)
  query=re.sub(' +',' ',query)

  # generating vector
  d=get_embedding_w2v(query.split()).reshape(1, -1)
  d=pca1.transform(d)
  vector = pca.transform(d)
  # ranking documents
  documents=testing_corpus[['docid','title','body']].copy()
  documents['similarity']=testing_corpus['vector'].apply(lambda x: cosine_similarity(np.array(vector).reshape(1, -1),np.array(x).reshape(1, -1)).item())
  documents['euclidean_distances']=testing_corpus['vector'].apply(lambda x: euclidean_distances(np.array(vector).reshape(1, -1),np.array(x).reshape(1, -1)).item())

  documents.sort_values(by='similarity',ascending=False,inplace=True)

  return documents.head(10).reset_index(drop=True)

In [ ]:
ranking_ir('michael jordan')

,docid,title,body,similarity,euclidean_distances
0,D2962495,How Much Is Air Jordan Net Worth,"Profession: Basketball player, Actor, Athlete,...",0.972593,0.000072
1,D3140874,Michael Jordan Net Worth,"Michael Jordan Net Worth· In Athletes, NBAMich...",0.853485,0.000082
2,D1520298,Whatâs Michael Jordan Doing Now â 2018 Update,Sports What’s Michael Jordan Doing Now – 2018 ...,0.779287,0.000092
3,D2785629,What is the name of a basketball team that is ...,Answers.com ® Wiki Answers ® Categories Entert...,0.764134,0.000082
4,D1142830,The 23 Best Air Jordan sneakers of All-Time,"December 20, 20144 Comments 818.2K Views The 2...",0.754769,0.000080
5,D1415956,Michael Jordan Biography,Quick Facts Name Michael Jordan Occupation Fam...,0.747657,0.000094
6,D2986655,Michael Jordan Biography,Quick Facts Name Michael Jordan Occupation Fam...,0.746982,0.000094
7,D1900195,What year did Michael Jordan play baseball?,Answers.com ® Wiki Answers ® Categories Entert...,0.722767,0.000085
8,D207390,Michael Jordan: King Of The Court,Michael Jordan: King Of The Court Length: 1071...,0.712568,0.000093
9,D721157,When Michael Jordan scored the most points in ...,Answers.com ® Wiki Answers ® Categories Entert...,0.677675,0.000088


In [ ]:
ranking_ir('nba championship michael jordan')

,docid,title,body,similarity,euclidean_distances
0,D842657,How many NBA championships did Michael Jordan ...,Answers.com ® Wiki Answers ® Categories Sports...,0.780235,0.000048
1,D1520298,Whatâs Michael Jordan Doing Now â 2018 Update,Sports What’s Michael Jordan Doing Now – 2018 ...,0.708433,0.000055
2,D2796474,How many championships does Michael Jordan have?,Jim Schiller 1 Contribution How many champions...,0.704772,0.000048
3,D207390,Michael Jordan: King Of The Court,Michael Jordan: King Of The Court Length: 1071...,0.693215,0.000055
4,D2986655,Michael Jordan Biography,Quick Facts Name Michael Jordan Occupation Fam...,0.687460,0.000056
5,D1415956,Michael Jordan Biography,Quick Facts Name Michael Jordan Occupation Fam...,0.685560,0.000056
6,D3140874,Michael Jordan Net Worth,"Michael Jordan Net Worth· In Athletes, NBAMich...",0.678341,0.000051
7,D2962495,How Much Is Air Jordan Net Worth,"Profession: Basketball player, Actor, Athlete,...",0.668931,0.000049
8,D848759,What Numbers Did Michael Jordan Wear?,Sports & Active Lifestyle Classic Sports Baske...,0.660229,0.000053
9,D2785629,What is the name of a basketball team that is ...,Answers.com ® Wiki Answers ® Categories Entert...,0.631000,0.000051


In [ ]:
# Function returning vector reperesentation of a document
def get_embedding_w2v(doc_tokens):
    embeddings = []
    if len(doc_tokens)<1:
        return np.zeros(300)
    else:
        for tok in doc_tokens:
            if tok in w2v_model.wv.key_to_index  :

                embeddings.append(w2v_model.wv.get_vector(tok))
            else:
                embeddings.append(np.random.rand(300))
        # mean the vectors of individual words to get the vector of the document
        return np.mean(embeddings, axis=0)

# Getting Word2Vec Vectors for Testing Corpus and Queries
testing_corpus['vector']=testing_corpus['lemmatized'].apply(lambda x :get_embedding_w2v(x.split()))
testing_queries['vector']=testing_queries['cleaned'].apply(lambda x :get_embedding_w2v(x.split()))

In [ ]:
a = np.zeros((testing_corpus['vector'].shape[0],testing_corpus['vector'][0].shape[0]))
for i in range(len(testing_corpus['vector'])):
    a[i,:] = testing_corpus['vector'][i]

b = np.zeros((testing_queries['vector'].shape[0],testing_queries['vector'][0].shape[0]))
for i in range(len(testing_queries['vector'])):
    b[i,:] = testing_queries['vector'][i]

from sklearn.decomposition import PCA
from sklearn.decomposition import FactorAnalysis

pca = FactorAnalysis(n_components=290)
testing_corpus1 = pca.fit_transform(a)
#pca1 = PCA(n_components = 300)
testing_queries1 = pca.transform(b)

In [ ]:
for i in range(len(testing_corpus['vector'])):
     testing_corpus['vector'][i]=testing_corpus1[i]
for i in range(len(testing_queries['vector'])):
     testing_queries['vector'][i]=testing_queries1[i]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances
# Function for calculating average precision for a query
def average_precision(qid,qvector):

  # Getting the ground truth and document vectors
  qresult=testing_result.loc[testing_result['qid']==qid,['docid','rel']]
  qcorpus=testing_corpus.loc[testing_corpus['docid'].isin(qresult['docid']),['docid','vector']]
  qresult=pd.merge(qresult,qcorpus,on='docid')

  # Ranking documents for the query
  qresult['similarity']=qresult['vector'].apply(lambda x: cosine_similarity(np.array(qvector).reshape(1, -1),np.array(x).reshape(1, -1)).item())
  qresult['euclidean_distances']=qresult['vector'].apply(lambda x: euclidean_distances(np.array(qvector).reshape(1, -1),np.array(x).reshape(1, -1)).item())

  qresult.sort_values(by='similarity',ascending=False,inplace=True)

  # Taking Top 10 documents for the evaluation
  ranking=qresult.head(10)['rel'].values

  # Calculating precision
  precision=[]
  for i in range(1,11):
    if ranking[i-1]:
      precision.append(np.sum(ranking[:i])/i)

  # If no relevant document in list then return 0
  if precision==[]:
    return 0

  return np.mean(precision)

# Calculating average precision for all queries in the test set
testing_queries['AP']=testing_queries.apply(lambda x: average_precision(x['qid'],x['vector']),axis=1)

# Finding Mean Average Precision
print('Mean Average Precision=>',testing_queries['AP'].mean())

Mean Average Precision=> 0.7975119901108588


In [ ]:
def ranking_ir(query):

  # pre-process Query
  query=query.lower()
  query=expand_contractions(query)
  query=clean_text(query)
  query=re.sub(' +',' ',query)

  # generating vector
  d=get_embedding_w2v(query.split()).reshape(1, -1)
  #d=pca1.transform(d)
  vector = pca.transform(d)
  # ranking documents
  documents=testing_corpus[['docid','title','body']].copy()
  documents['similarity']=testing_corpus['vector'].apply(lambda x: cosine_similarity(np.array(vector).reshape(1, -1),np.array(x).reshape(1, -1)).item())
  documents['euclidean_distances']=testing_corpus['vector'].apply(lambda x: euclidean_distances(np.array(vector).reshape(1, -1),np.array(x).reshape(1, -1)).item())

  documents.sort_values(by='similarity',ascending=False,inplace=True)

  return documents.head(10).reset_index(drop=True)

In [ ]:
bm=ranking_ir('michael jordan')
bm

,docid,title,body,similarity,euclidean_distances
0,D2962495,How Much Is Air Jordan Net Worth,"Profession: Basketball player, Actor, Athlete,...",0.966280,49.994113
1,D3140874,Michael Jordan Net Worth,"Michael Jordan Net Worth· In Athletes, NBAMich...",0.860316,56.594991
2,D1520298,Whatâs Michael Jordan Doing Now â 2018 Update,Sports What’s Michael Jordan Doing Now – 2018 ...,0.766336,64.198021
3,D2785629,What is the name of a basketball team that is ...,Answers.com ® Wiki Answers ® Categories Entert...,0.759959,58.002952
4,D2986655,Michael Jordan Biography,Quick Facts Name Michael Jordan Occupation Fam...,0.749742,65.150028
5,D1415956,Michael Jordan Biography,Quick Facts Name Michael Jordan Occupation Fam...,0.749027,65.167490
6,D848759,What Numbers Did Michael Jordan Wear?,Sports & Active Lifestyle Classic Sports Baske...,0.739552,62.105476
7,D207390,Michael Jordan: King Of The Court,Michael Jordan: King Of The Court Length: 1071...,0.737504,64.905176
8,D1142830,The 23 Best Air Jordan sneakers of All-Time,"December 20, 20144 Comments 818.2K Views The 2...",0.736993,58.116623
9,D3492208,How many NBA championships did Michael Jordan ...,Flyinryan 16 Contributions How many NBA champi...,0.733345,57.456273


In [ ]:
ranking_ir('nba championship michael jordan')

,docid,title,body,similarity,euclidean_distances
0,D842657,How many NBA championships did Michael Jordan ...,Answers.com ® Wiki Answers ® Categories Sports...,0.908030,34.127627
1,D2796474,How many championships does Michael Jordan have?,Jim Schiller 1 Contribution How many champions...,0.877959,33.335328
2,D1415956,Michael Jordan Biography,Quick Facts Name Michael Jordan Occupation Fam...,0.828425,45.986487
3,D2986655,Michael Jordan Biography,Quick Facts Name Michael Jordan Occupation Fam...,0.828419,45.977738
4,D1520298,Whatâs Michael Jordan Doing Now â 2018 Update,Sports What’s Michael Jordan Doing Now – 2018 ...,0.824761,45.248978
5,D2962495,How Much Is Air Jordan Net Worth,"Profession: Basketball player, Actor, Athlete,...",0.813277,37.972066
6,D3492208,How many NBA championships did Michael Jordan ...,Flyinryan 16 Contributions How many NBA champi...,0.802346,37.951125
7,D207390,Michael Jordan: King Of The Court,Michael Jordan: King Of The Court Length: 1071...,0.781068,46.223687
8,D3140874,Michael Jordan Net Worth,"Michael Jordan Net Worth· In Athletes, NBAMich...",0.778730,41.173653
9,D2785629,What is the name of a basketball team that is ...,Answers.com ® Wiki Answers ® Categories Entert...,0.773027,40.232206
